In [17]:
import main
from os import listdir
import pandas as pd
import TAtT
import Christofides
import time as T
import tracemalloc

In [18]:

df = pd.DataFrame({'Nome':['a'], 'Tamanho':[0], 'BnB':[0], 'timeBnB':[0], 'spaceBnB':[0], 'TAtT':[0], 'timeTAtT':[0], 'spaceTAtT':[0], 'Christofides':[0], 'timeChristofides':[0], 'spaceChristofides':[0]})

In [20]:
import signal
import time as T
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


In [ ]:
# newrow = pd.DataFrame({'Name':['brd14051.tsp'],'Val':[None],'time':[None],'space':[None]})


In [21]:
aux = pd.read_csv('CSVbackup/TAtT_backup.csv')
# aux = aux['Name']
# aux = aux[aux != 'a']
# aux = pd.concat([aux,newrow],ignore_index=True)
# aux = aux.loc[:, ~aux.columns.str.contains('^Unnamed')]
# aux.to_csv('CSVbackup/TAtT_backup.csv')
files = list(listdir('ALL_tsp'))
files = [file for file in files if file not in aux.values]
print(len(files))
aux

41


,Unnamed: 0,Name,Val,time,space
0,0,a,0.000000,0.00000,0.0
1,1,pr439.tsp,146846.650609,8.82000,195056.0
2,2,rd100.tsp,10791.655816,0.09000,102480.0
3,3,rl5934.tsp,842363.531789,798.65000,269048.0
4,4,pcb442.tsp,69762.396742,1.34000,165274.0
5,5,u2319.tsp,405732.655793,29.94000,184296.0
6,6,gil262.tsp,3358.154177,0.41000,157598.0
7,7,pcb3038.tsp,196718.243582,66.37000,231544.0
8,8,lin105.tsp,20044.571503,0.07000,94742.0
9,9,fl417.tsp,16299.726150,0.99000,166704.0


In [ ]:
# files = list(listdir('ALL_tsp'))
dfTAtT = pd.DataFrame({'Name':['a'],'Val':[0],'time':[0],'space':[0]})
for file in files:
    try:
        with time_limit(1800):
            name = file
            print(name)
            (G,_,_) = main.Graph('ALL_tsp/'+file)
            init = T.time()
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()
            (_, W) = TAtT.TatT(G,1)
            snapshot2 = tracemalloc.take_snapshot()
            tracemalloc.stop()
            diferenca = snapshot2.compare_to(snapshot1, 'lineno')
            space = round(sum(abs(stat.size_diff) for stat in diferenca), 2)
            time = T.time() - init
            newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[round(time,2)],'space':[space]})
            dfTAtT = pd.concat([dfTAtT,newrow],ignore_index=True)
            dfTAtT.to_csv('CSV/TAtT.csv')
            print('add ',name,W,time,space)
    except TimeoutException as e:
        newrow = pd.DataFrame({'Name':[file],'Val':[None],'time':[1800],'space':[None]})
        dfTAtT = pd.concat([dfTAtT,newrow],ignore_index=True)
        dfTAtT.to_csv('CSV/TAtT.csv')
        print("Timed out! ",file)
dfTAtT = dfTAtT.drop([0], axis=0)


a280.tsp
add  a280.tsp 3751.3988440001685 0.4531400203704834 184347
d1291.tsp
add  d1291.tsp 80210.59377217208 10.402706146240234 202256
pr264.tsp
add  pr264.tsp 70613.7888411385 0.4085109233856201 158918
pr76.tsp
add  pr76.tsp 145338.11445183848 0.02717113494873047 48960
d493.tsp
add  d493.tsp 45571.71848892038 1.762117862701416 192632
pr136.tsp
add  pr136.tsp 151913.74200165656 0.0968179702758789 112328
rat195.tsp
add  rat195.tsp 3329.6659014343 0.20484614372253418 148696
rl11849.tsp


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10683f620>>
Traceback (most recent call last):
  File "/Users/izadoraganem/Library/Python/3.13/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
  File "/var/folders/z6/xyw2mf3x1l96q4_w6y22mq0r0000gn/T/ipykernel_52448/1868677448.py", line 10, in signal_handler
TimeoutException: Timed out!


In [ ]:
# files = list(listdir('ALL_tsp'))
dfChristofides = pd.DataFrame({'Name':['a'],'Val':[0],'time':[0],'space':[0]})
for file in files:
    try:
        with time_limit(65):
            name = file
            print(name)
            (G,_,_) = main.Graph('ALL_tsp/'+file)
            init = T.time()
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()
            (_, W) = Christofides.Christofides(G,1)
            time = T.time() - init
            snapshot2 = tracemalloc.take_snapshot()
            tracemalloc.stop()      
            diferenca = snapshot2.compare_to(snapshot1, 'lineno')
            space = round(sum(abs(stat.size_diff) for stat in diferenca), 2)
            newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[time],'space':[space]})
            dfChristofides = pd.concat([dfChristofides,newrow],ignore_index=True)
            dfChristofides.to_csv('CSV/Christofides.csv')
            print('add ',name,W,time,space)
    except TimeoutException as e:
        newrow = pd.DataFrame({'Name':[file],'Val':[None],'time':[30],'space':[None]})
        dfChristofides = pd.concat([dfChristofides,newrow],ignore_index=True)
        dfTAtT.to_csv('CSV/TAtT.csv')
        print("Timed out! ",file)
dfChristofides = dfChristofides.drop([0],axis=0)

d15112.tsp
Timed out!  d15112.tsp
pr107.tsp
add  pr107.tsp 57503.041524445405 0.16680216789245605 605382
kroB200.tsp
add  kroB200.tsp 38978.906995041165 1.0576579570770264 1302758
brd14051.tsp


KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 
